In [1]:
# !pip uninstall -y pyarrow
# !pip install pyarrow==15.0.2
# !pip install hopsworks==4.2.*
# !pip install pyspark

In [2]:
# !pip install sparkxgb

In [3]:
import hopsworks
import pandas as pd
from pyspark.sql import SparkSession
from sklearn.preprocessing import OneHotEncoder
from pyspark.sql.types import DoubleType
from pickle import dump
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from xgboost.spark import SparkXGBClassifier
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [4]:
project = hopsworks.login()
fs = project.get_feature_store()


Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1334425


In [5]:
final_data = fs.get_feature_group('final_data', version=1)

In [6]:
query = final_data.select_all()

In [7]:
final_merge = query.read(read_options={"use_hive": True})

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.12s) 


In [8]:
final_merge.head()

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,event_time_y,is_midnight
0,3725,10497416,R-2c5432ed,2019-02-10 07:00:00+00:00,2019-02-12 16:54:00+00:00,1,28.818182,6.727273,0.0,51.272727,...,Brandon Cruz,male,53,23,proactive,6,10497416,59.77,2026-01-04 16:45:05.210435+00:00,1
1,4299,28978466,R-f8640cff,2019-01-19 07:00:00+00:00,2019-01-19 12:44:24+00:00,0,46.000000,9.000000,0.0,75.000000,...,Brian Garcia,male,49,5,conservative,8,28978466,45.73,2026-01-04 16:45:05.210435+00:00,0
2,5674,13363106,R-07d2544a,2019-02-02 07:00:00+00:00,2019-02-04 03:04:48+00:00,0,49.888889,5.777778,0.0,67.444444,...,Joel Howard,male,52,17,conservative,7,13363106,52.25,2026-01-04 16:45:05.210435+00:00,1
3,9783,61984883,R-d87e53cd,2019-02-09 07:00:00+00:00,2019-02-09 20:38:24+00:00,0,80.500000,10.000000,0.0,63.500000,...,Jerry Powers,male,41,7,conservative,2,61984883,56.94,2026-01-04 16:45:05.210435+00:00,0
4,4088,18855810,R-c061582f,2019-02-06 07:00:00+00:00,2019-02-06 19:22:48+00:00,0,22.000000,10.000000,0.0,73.500000,...,John Vasquez,male,47,4,conservative,4,18855810,48.03,2026-01-04 16:45:05.210435+00:00,0


In [9]:
final_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 51 columns):
 #   Column                          Non-Null Count  Dtype                  
---  ------                          --------------  -----                  
 0   unique_id                       12308 non-null  int64                  
 1   truck_id                        12308 non-null  int64                  
 2   route_id                        12308 non-null  object                 
 3   departure_date                  12308 non-null  datetime64[us, Etc/UTC]
 4   estimated_arrival               12308 non-null  datetime64[us, Etc/UTC]
 5   delay                           12308 non-null  int64                  
 6   route_avg_temp                  12308 non-null  float64                
 7   route_avg_wind_speed            12308 non-null  float64                
 8   route_avg_precip                12308 non-null  float64                
 9   route_avg_humidity              12308 n

In [10]:
final_merge[final_merge['origin_temp'].isnull()]

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,event_time_y,is_midnight
559,7661,18091756,R-112b790b,2019-01-25 07:00:00+00:00,2019-01-27 02:40:48+00:00,1,66.555556,6.888889,0.000000,90.888889,...,Neil Herring,male,45,7,proactive,3,18091756,58.02,2026-01-04 16:45:05.210435+00:00,1
3782,8163,24746768,R-b5f9418a,2019-01-25 07:00:00+00:00,2019-01-27 14:35:24+00:00,0,47.454545,9.090909,0.000000,70.636364,...,William Anderson III,male,50,0,conservative,4,24746768,40.69,2026-01-04 16:45:05.210435+00:00,1
8379,11360,22916520,R-78ee1f97,2019-01-25 07:00:00+00:00,2019-01-28 10:08:24+00:00,0,57.500000,10.142857,0.000000,78.214286,...,Thomas Ochoa,male,57,19,proactive,6,22916520,63.64,2026-01-04 16:45:05.210435+00:00,1
12077,7721,24654257,R-21472caf,2019-01-25 07:00:00+00:00,2019-01-27 16:50:24+00:00,0,69.000000,12.363636,0.018182,79.181818,...,Marc Walters,male,47,5,proactive,3,24654257,61.93,2026-01-04 16:45:05.210435+00:00,1


In [11]:
routes_data = fs.get_feature_group('routes_details_fg', version=1)

routes_data_query = routes_data.select_all()

routes_df = routes_data_query.read(read_options={"use_hive": True})

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.43s) 


In [12]:
routes_df[routes_df.route_id.isin(['R-112b790b', 'R-78ee1f97','R-b5f9418a', 'R-21472caf'])]

,route_id,origin_id,destination_id,distance,average_hours,event_time
260,R-21472caf,C-f8f01604,C-2e349ccd,2892.14,57.84,2026-01-04 16:59:50.232103+00:00
398,R-b5f9418a,C-f8f01604,C-4fe0fa24,2779.33,55.59,2026-01-04 16:59:50.232103+00:00
400,R-112b790b,C-f8f01604,C-d3bb431c,2183.94,43.68,2026-01-04 16:59:50.232103+00:00
1357,R-78ee1f97,C-f8f01604,C-f5ed4c15,3757.02,75.14,2026-01-04 16:59:50.232103+00:00


In [13]:
weather_data = fs.get_feature_group('city_weather_details_fg', version=1)

weather_query = weather_data.select_all()

weather_df = weather_query.read(read_options={"use_hive": True})

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.85s) 


In [14]:
weather_df[(weather_df.city_id=='C-f8f01604')&(weather_df.date==pd.to_datetime('2019-01-25'))]

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder


In [15]:
final_merge = (
    final_merge
        .dropna(subset=[
            'origin_temp',
            'origin_wind_speed',
            'origin_precip',
            'origin_humidity',
            'origin_visibility',
            'origin_pressure'
        ])
)


In [16]:
spark = SparkSession.builder.getOrCreate()
final_merge = spark.createDataFrame(final_merge)

In [17]:
null_counts = final_merge.select(
    *[
        F.sum(F.col(c).isNull().cast("int")).alias(c)
        for c in final_merge.columns
    ]
)

In [18]:
cts_cols = [
    'route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip',
    'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure',
    'distance', 'average_hours',
    'origin_temp', 'origin_wind_speed', 'origin_precip', 'origin_humidity',
    'origin_visibility', 'origin_pressure',
    'destination_temp', 'destination_wind_speed', 'destination_precip',
    'destination_humidity', 'destination_visibility', 'destination_pressure',
    'avg_no_of_vehicles', 'truck_age', 'load_capacity_pounds',
    'mileage_mpg', 'age', 'experience', 'average_speed_mph'
]

cat_cols = [
    'route_description', 'origin_description', 'destination_description',
    'accident', 'fuel_type', 'gender', 'driving_style',
    'ratings', 'is_midnight'
]

target = ['delay']


In [19]:
final_merge.select(
    F.min("estimated_arrival").alias("min_estimated_arrival"),
    F.max("estimated_arrival").alias("max_estimated_arrival")
).show()

+---------------------+---------------------+
|min_estimated_arrival|max_estimated_arrival|
+---------------------+---------------------+
|  2019-01-01 07:04:48|  2019-02-14 16:06:00|
+---------------------+---------------------+



In [20]:
final_merge = final_merge.withColumn(
    "estimated_arrival",
    F.to_timestamp("estimated_arrival")
)

In [21]:
final_merge = final_merge.withColumn(
    "weather_severity",
    (
        F.coalesce(F.col("origin_precip"), F.lit(0)) +
        F.coalesce(F.col("destination_precip"), F.lit(0)) +
        (100 - F.coalesce(F.col("origin_visibility"), F.lit(100))) / 100 +
        (100 - F.coalesce(F.col("destination_visibility"), F.lit(100))) / 100 +
        F.coalesce(F.col("origin_wind_speed"), F.lit(0)) / 50 +
        F.coalesce(F.col("destination_wind_speed"), F.lit(0)) / 50
    )
)

In [22]:
final_merge = final_merge.withColumn(
    "route_difficulty",
    F.col("distance") * F.col("weather_severity")
)

In [23]:
final_merge = final_merge.withColumn(
    "traffic_exposure",
    F.col("avg_no_of_vehicles") * F.col("distance")
)

In [24]:
final_merge = final_merge.withColumn(
    "vehicle_stress",
    F.when(F.col("mileage_mpg") > 0,
           F.col("load_capacity_pounds") / F.col("mileage_mpg")
    ).otherwise(0)
)

In [25]:
final_merge = final_merge.withColumn(
    "driver_risk",
    F.when(F.col("experience") > 0,
           (1 / F.col("experience")) *
           F.col("average_speed_mph") *
           F.col("weather_severity")
    ).otherwise(0)
)

In [26]:
final_merge = final_merge.withColumn(
    "night_visibility_risk",
    F.col("is_midnight").cast("int") *
    (100 - F.coalesce(F.col("origin_visibility"), F.lit(100)))
)

In [27]:
final_merge = final_merge.withColumn(
    "distance_bucket",
    F.when(F.col("distance") < 100, 0)
     .when(F.col("distance") < 300, 1)
     .when(F.col("distance") < 600, 2)
     .otherwise(3)
)


In [28]:
final_merge = final_merge.withColumn(
    "env_load",
    F.col("weather_severity") *
    F.col("avg_no_of_vehicles") *
    F.col("distance")
)

In [29]:
final_merge = final_merge.withColumn(
    "exp_bucket",
    F.when(F.col("experience") < 2, 0)
     .when(F.col("experience") < 5, 1)
     .when(F.col("experience") < 10, 2)
     .otherwise(3)
)

In [30]:
final_merge = final_merge.withColumn(
    "speed_bucket",
    F.when(F.col("average_speed_mph") < 30, 0)
     .when(F.col("average_speed_mph") < 50, 1)
     .when(F.col("average_speed_mph") < 70, 2)
     .otherwise(3)
)


In [31]:
final_merge = final_merge.withColumn(
    "visibility_bucket",
    F.when(F.col("origin_visibility") < 30, 0)
     .when(F.col("origin_visibility") < 60, 1)
     .otherwise(2)
)

In [32]:
final_merge = final_merge.withColumn(
    "speed_traffic_risk",
    F.col("average_speed_mph") * F.col("avg_no_of_vehicles")
)

In [33]:
new_cols = [
    "distance_bucket",
    "speed_bucket",
    "visibility_bucket",
    "speed_traffic_risk",
    "env_load",
    "exp_bucket",
    "weather_severity",
    "route_difficulty",
    "driver_risk",
    "vehicle_stress",
    "traffic_exposure",
    "night_visibility_risk"
]

cts_cols = cts_cols + new_cols

In [34]:
cutoff_train = "2019-01-30 00:00:00"
cutoff_val   = "2019-02-07 00:00:00"

In [35]:
train_df = final_merge.filter(
    F.col("estimated_arrival") <= F.lit(cutoff_train)
)

validation_df = final_merge.filter(
    (F.col("estimated_arrival") > F.lit(cutoff_train)) &
    (F.col("estimated_arrival") <= F.lit(cutoff_val))
)

test_df = final_merge.filter(
    F.col("estimated_arrival") > F.lit(cutoff_val)
)


In [36]:
final_merge = final_merge.withColumn(
    "estimated_arrival",
    F.to_utc_timestamp("estimated_arrival", "UTC")
)


In [37]:
for c in cts_cols:
    final_merge = final_merge.withColumn(c, F.col(c).cast(DoubleType()))

In [38]:
indexers = [
    StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep")
    for c in cat_cols
]

feature_cols = cts_cols + [c+"_idx" for c in cat_cols]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

pipeline = Pipeline(stages=indexers + [assembler])
pipe_model = pipeline.fit(train_df)

train_df = pipe_model.transform(train_df)
validation_df = pipe_model.transform(validation_df)
test_df = pipe_model.transform(test_df)


In [39]:
train_df = train_df.select("features", F.col("delay").alias("label"))
validation_df = validation_df.select("features", F.col("delay").alias("label"))
test_df = test_df.select("features", F.col("delay").alias("label"))

In [40]:
pos = train_df.filter("label = 1").count()
neg = train_df.filter("label = 0").count()

scale = neg / pos

In [41]:
xgb = SparkXGBClassifier(
    features_col="features",
    label_col="label",
    prediction_col="prediction",
    probability_col="probability",

    # Core learning setup
    tree_method="hist",
    num_boost_round=800,
    eta=0.03,
    max_depth=12,

    # Sampling
    subsample=0.8,
    colsample_bytree=0.7,
    colsample_bylevel=0.7,

    # Regularization (prevents overfit with deep trees)
    min_child_weight=5,
    gamma=1.5,
    reg_alpha=0.5,
    reg_lambda=1.5,

    # Class imbalance
    scale_pos_weight=scale
)

In [42]:
xgb_model = xgb.fit(train_df)

INFO:XGBoost-PySpark:Running xgboost-3.1.3 on 1 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bylevel': 0.7, 'colsample_bytree': 0.7, 'device': 'cpu', 'gamma': 1.5, 'max_depth': 12, 'min_child_weight': 5, 'reg_alpha': 0.5, 'reg_lambda': 1.5, 'scale_pos_weight': 2.093926820067899, 'subsample': 0.8, 'tree_method': 'hist', 'eta': 0.03, 'nthread': 1}
	train_call_kwargs_params: {'num_boost_round': 100, 'verbose_eval': True}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Finished xgboost training!


In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

preds = xgb_model.transform(test_df)

evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="probability",
    metricName="areaUnderROC"
)

print("AUC:", evaluator.evaluate(preds))


AUC: 0.8136151054704938


In [44]:
pdf = preds.select("label", "probability").toPandas()
pdf["prob_1"] = pdf["probability"].apply(lambda x: float(x[1]))

In [45]:
from sklearn.metrics import roc_curve, auc
import plotly.express as px

fpr, tpr, _ = roc_curve(pdf["label"], pdf["prob_1"])
roc_auc = auc(fpr, tpr)

fig = px.line(x=fpr, y=tpr, title=f"ROC Curve (AUC={roc_auc:.3f})")
fig.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)
fig.show()

In [46]:
preds = xgb_model.transform(test_df).select(
    "label", "prediction", "probability"
)

In [47]:
pdf = preds.toPandas()
pdf["prob_1"] = pdf["probability"].apply(lambda x: float(x[1]))

In [48]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score

best_f1 = 0
best_t = 0

for t in np.arange(0.1, 0.9, 0.01):
    preds_t = (pdf["prob_1"] >= t).astype(int)
    f1 = f1_score(pdf["label"], preds_t)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

best_t, best_f1

(0.43999999999999984, 0.7520927237604637)

In [49]:
final_preds = (pdf["prob_1"] >= best_t).astype(int)

precision = precision_score(pdf["label"], final_preds)
recall = recall_score(pdf["label"], final_preds)
f1 = f1_score(pdf["label"], final_preds)

precision, recall, f1

(0.776595744680851, 0.7290886392009988, 0.7520927237604637)

In [51]:
fpr, tpr, thresholds = roc_curve(pdf["label"], pdf["prob_1"])
roc_auc = auc(fpr, tpr)

fig = px.line(
    x=fpr, y=tpr,
    title=f"ROC Curve (AUC = {roc_auc:.3f})",
    labels={"x": "False Positive Rate", "y": "True Positive Rate"}
)

fig.add_shape(type="line", x0=0, y0=0, x1=1, y1=1)

fig.show()

In [52]:
ths = np.arange(0.05, 0.95, 0.01)

f1s, precisions, recalls = [], [], []

for t in ths:
    preds_t = (pdf["prob_1"] >= t).astype(int)
    f1s.append(f1_score(pdf["label"], preds_t))
    precisions.append(precision_score(pdf["label"], preds_t))
    recalls.append(recall_score(pdf["label"], preds_t))

In [53]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=ths, y=f1s, name="F1 Score"))
fig.add_trace(go.Scatter(x=ths, y=precisions, name="Precision"))
fig.add_trace(go.Scatter(x=ths, y=recalls, name="Recall"))

fig.update_layout(
    title="F1, Precision, Recall vs Threshold",
    xaxis_title="Threshold",
    yaxis_title="Score"
)

fig.show()